# NGsolve example - wound field rotor

In [ ]:
%matplotlib ipympl
import matplotlib.pyplot as plt
import numpy as np
import sys, os
sys.path.append(os.path.relpath("../ngsolve/src"))
import hiped as hp
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.geom2d import SplineGeometry
import time
from IPython.display import clear_output
from copy import deepcopy

# 1) Geometry

In [ ]:
def poleMotor(Npp = 2, Rrmin=0.01, Rrmax = 0.0495, Rsmin = 0.0505 ,Rsmax = 0.1, maxh=0.1):
    geo = SplineGeometry()
    pref = [(1,0),(1,tan(pi/(2*Npp))),(cos(2*pi/(2*Npp)),sin(2*pi/(2*Npp)))]
    P1= [ geo.AppendPoint(pnt[0]*Rrmin, pnt[1]*Rrmin) for pnt in pref ]
    P2= [ geo.AppendPoint(pnt[0]*Rrmax, pnt[1]*Rrmax) for pnt in pref ]
    P3= [ geo.AppendPoint(pnt[0]*Rsmin, pnt[1]*Rsmin) for pnt in pref ]
    P4= [ geo.AppendPoint(pnt[0]*Rsmax, pnt[1]*Rsmax) for pnt in pref ]
    
    geo.Append(['spline3',P1[0],P1[1],P1[2]], leftdomain=0, rightdomain=1, bc="a0", maxh = maxh)
    geo.Append(['spline3',P2[0],P2[1],P2[2]], leftdomain=1, rightdomain=2, bc="e1", maxh = maxh/3)
    geo.Append(['spline3',P3[0],P3[1],P3[2]], leftdomain=2, rightdomain=3, bc="e2", maxh = maxh/3)
    geo.Append(['spline3',P4[0],P4[1],P4[2]], leftdomain=3, rightdomain=0, bc="a0", maxh = maxh)
    
    y1 = geo.Append(['line',P1[0],P2[0]], leftdomain=1, rightdomain=0, bc="antiPeriodic")
    geo.Append(['line',P1[2],P2[2]], leftdomain=0, rightdomain=1, bc="antiPeriodic", copy = y1)
    y2 = geo.Append(['line',P2[0],P3[0]], leftdomain=2, rightdomain=0, bc="antiPeriodic")
    geo.Append(['line',P2[2],P3[2]], leftdomain=0, rightdomain=2, bc="antiPeriodic", copy = y2)
    y3 = geo.Append(['line',P3[0],P4[0]], leftdomain=3, rightdomain=0, bc="antiPeriodic")
    geo.Append(['line',P3[2],P4[2]], leftdomain=0, rightdomain=3, bc="antiPeriodic", copy = y3)

    geo.SetMaterial(1,"Rotor"); geo.SetDomainMaxH(1,maxh/2)
    geo.SetMaterial(2,"Airgap") ; geo.SetDomainMaxH(2,maxh/3)
    geo.SetMaterial(3,"Stator")
    mesh=Mesh(geo.GenerateMesh(maxh=maxh, grading = 0.05)).Curve(3)
    return mesh

mesh = poleMotor(Npp=4, maxh = 0.004)
Draw(mesh)

# 2) Material definition

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

#  inspired from Altair Flux model, can also be coupled with temperature : 
# https://help.altair.com/flux/Flux/Help/francais/UserGuide/Francais/topics/CourbeDeSaturationAnalytiqueControleDuCoudeFonctionExponentielleDeT.htm

NU = [0]

def setIron(mur=1000, K=2, n=20):
    def h(b):
        return 1/(4e-7*np.pi) * b + ( 1/(mur*4e-7*np.pi) - 1/(4e-7*np.pi) )*K*b/(K**n + b**n)**(1/n)
    
    def nu(b):
        return (h(b) / b).Compile()
    #def nu(b):
    #    return 1/(mur*4e-7*np.pi)
    
    NU[0] = nu
    
    B = np.linspace(0, 2, 100)
    H = h(B)
    fig.clf()
    plt.plot(H, B, label = "Iron")
    plt.plot(B/(4e-7*pi), B, label = "Air")
    plt.axis([0, np.max(H), 0, np.max(B)])
    plt.grid(); plt.legend(loc = 'best')
    plt.xlabel("H (A/m)"); plt.ylabel("B (T)")
    plt.show()
    
fig = plt.figure()
mur = widgets.FloatSlider(min=100., max=1e4, step=1e2, value=8e3)
K = widgets.FloatSlider(min=1., max=4., step=0.1, value=2.)
n = widgets.FloatSlider(min=5., max=50., step=1., value=20)
_ = interact(setIron, mur=mur, K=K, n=n)


# 3) Interpolation of physical properties

In [ ]:
dom = hp.Domain(3) 

# Current Density

J = 10e6
penalJ = hp.Penalization("simp", 1, reverse=False)

jDCp = hp.VertexFunction(label = "jDCp", f = lambda u : CF(J),     dfdu = lambda u : CF(0),    flagNGSolve = True)
jDCm = hp.VertexFunction(label = "jDCm", f = lambda u : CF(-J),    dfdu = lambda u : CF(0),  flagNGSolve = True)
jiron = hp.VertexFunction(label = "jiron", f = lambda u : CF(0),     dfdu = lambda u : CF(0), flagNGSolve = True)

interpJ = hp.Interpolation(dom, [jDCp,jDCm,jiron], label = "rotor", penalization= penalJ)

# Reluctivity

nu0 = 1/(4e-7*np.pi)
penalNu = hp.Penalization("simp", 1, reverse=False)

nuDCp = hp.VertexFunction(label = "nuDCp", f = lambda b : nu0 ,     dfdu = lambda u : CF(0),    flagNGSolve = True)
nuDCm = hp.VertexFunction(label = "nuDCm", f = lambda b : nu0 ,     dfdu = lambda u : CF(0),  flagNGSolve = True)
nuIron = hp.VertexFunction(label = "nuIron", f = lambda b : NU[0](Norm(b)),     dfdu = lambda b :  0, flagNGSolve = True) 
# nuIron derivative false because we will use automatic differentiation

interpNu =  hp.Interpolation(dom, [nuDCp,nuDCm,nuIron], label = "rotor", penalization= penalNu)

fesRho = L2(mesh, definedon = "Rotor")
#fesRho = H1(mesh, definedon = "Rotor")

rho = interpNu.setInitialVariable(typeInit = "zero", NGSpace = fesRho) 
rho = interpNu.projection(rho)
plt.figure()
interpNu.plot(rho)
plt.show()
#plt.figure()
#interpJ.plot(rho)
#plt.show()

# 4) Solver

In [ ]:
from ngsolve.solvers import Newton

nu0 = 1/(4e-7*pi)
fes = Periodic(H1(mesh, dirichlet = "a0"),[-1,-1,-1])

def weakForm(a,aStar,rho):
    w, dwdx = interpNu.evalBasisFunction(rho)
    b = Norm(grad(a)+1e-6*CF((1,1)))
    wf = (grad(aStar) * interpNu.eval(rho, b, w) * grad(a)) * dx("Rotor")
    wf += (grad(aStar) * nu0 * grad(a)) * dx("Stator|Airgap")
    wf += - interpJ.eval(rho, b, w) * aStar * dx("Rotor")
    return  wf

def solveState(rho, aInit = GridFunction(fes)):
    a, aStar = fes.TnT()
    bf = BilinearForm(fes)
    bf += weakForm(a,aStar,rho)
    Newton(bf, aInit, freedofs=fes.FreeDofs(), maxit=100, maxerr=1e-8, inverse="pardiso",dampfactor=0.7, printing=False)
    return aInit
           
directState = solveState(rho)
Draw(directState,mesh)

# 4) Objective function and adjoint

In [ ]:
areaAirgap = Integrate(CF(1) * dx("Airgap"), mesh)
perimeterAirgap = Integrate(CF(1) * ds("e1"), mesh)
thicknessAirgap = areaAirgap/perimeterAirgap

ur = Normalize(CF((x,y)))
R = CF(((0,1),(-1,0)), dims = (2,2))

def Jobj(a):
    return Integrate((R*grad(a))*ur*dx("Airgap"), mesh)/thicknessAirgap

def dJobj(aStar):
    return (R*grad(aStar))*ur*dx("Airgap")

def solveAdjoint(a,rho):
    fes = Periodic(H1(mesh, dirichlet = "a0"), [-1,-1,-1])
    p, aStar = fes.TnT()
    bf = BilinearForm(fes)
    bf += weakForm(p,aStar,rho)
    bf.AssembleLinearization(a.vec)
    rhs = LinearForm(dJobj(aStar))
    rhs.Assemble()
    gfu = GridFunction(fes)
    gfu.vec.FV().NumPy()[fes.FreeDofs()] = np.random.rand(np.sum(fes.FreeDofs()))
    gfu.vec.data = -1* bf.mat.Inverse(fes.FreeDofs(), inverse="sparsecholesky") * rhs.vec
    return gfu

p = solveAdjoint(directState,rho)
Draw(p)

# 5) Gradient

In [ ]:
def gradRho(u,p,rho):
    b = Norm(grad(u))
    w, dwdx = interpNu.evalBasisFunction(rho)
    dnudrho = interpNu.evaldx(rho,b,w,dwdx)
    dJdrho = interpJ.evaldx(rho,b,w,dwdx)
    g = rho.copy()
    keys = w.keys()
    for k in keys:
        g[k] = [(grad(u) * (dnudrho[k][i] * grad(p)))  - dJdrho[k][i] * p for i in range(len(dnudrho[k]))]
    return g

In [ ]:
## Initialisation

alpha = 0.1       # Pas initial
alpha_min = 1e-4  # Pas minimal
n_max = 100        # Nombre d'itérations de l'algorithme d'optimisation
n = 0

# Définition de la situation initiale

objectiveHistory = [Jobj(directState)]
rhoHistory = [deepcopy(rho)]

In [ ]:
## Boucle d'optimisation (descente de gradient avec contrôle du pas)
failureLineSearch = False
while( n < n_max and alpha > alpha_min):
    
    if failureLineSearch:
        # 1) State :
        directState = solveState(rho)
    else:
        # 2) Adjoint :
        adjointState = solveAdjoint(directState,rho)
    
        # 3) Gradient computation :
        gradient = gradRho(directState,adjointState,rho)
        normG = GridFunction(fesRho)
        normG.Set(sqrt(gradient['rotor'][0]**2 +gradient['rotor'][1]**2) )
    
    # 4) Mise à jour :
    rho_test = deepcopy(rho)
    for i in range(len(rho['rotor'])):
        rho_test['rotor'][i].Set(rho['rotor'][i] - alpha * gradient['rotor'][i]/normG) 
    n += 1
    
    # 5) Projection :
    
    rho_test = interpNu.projection(rho_test)
    
    # 6) Contrôle du pas :
    a_test = solveState(rho_test)
    objectiveHistory.append(Jobj(a_test))
    clear_output(wait = True)
    
    print(f'it n°{n} | f = {objectiveHistory[-1]} | step = {alpha}')
    
    if objectiveHistory[-1] >= objectiveHistory[-2]:
        alpha = alpha/2
        objectiveHistory.pop()
        failureLineSearch = True
    elif objectiveHistory[-1] < objectiveHistory[-2]:
        alpha = alpha*1.2
        rho = rho_test
        rhoHistory.append(deepcopy(rho))
        failureLineSearch = False

In [ ]:
w, dwdx = interpNu.evalBasisFunction(rhoHistory[-1])
Draw(w['rotor'][0], mesh)
Draw(R*grad(directState), mesh)

In [ ]:
plt.figure()
plt.plot(objectiveHistory)
plt.show()

In [ ]:
plt.figure()
interpNu.plot(rhoHistory[-1])
plt.show()